# AirBNB data exploration

In this notebook, we're going to try to answer the following questions:

- What is affecting price variations?
- Does it cares that a host has several listings on price variations?
- When is a property on high or low price? (predictable)

In [1]:
# Required imports for the project
import pandas as pd
import numpy as np
import seaborn as sns
import swifter


from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

from matplotlib import pyplot as plt
import matplotlib

#Custom function to download datasets from airbnb site.
from utils import download_datasets

In [11]:
#datasets to download
datasets_urls_boston = [
    'http://data.insideairbnb.com/united-states/ma/boston/2018-08-17/data/listings.csv.gz',
    'http://data.insideairbnb.com/united-states/ma/boston/2018-08-17/data/calendar.csv.gz'
]

datasets_urls_seattle = [
    'http://data.insideairbnb.com/united-states/wa/seattle/2018-08-16/data/listings.csv.gz',
    'http://data.insideairbnb.com/united-states/wa/seattle/2018-08-16/data/calendar.csv.gz'
]
datasets_urls_austin = [
    'http://data.insideairbnb.com/united-states/tx/austin/2018-08-14/data/listings.csv.gz',
    'http://data.insideairbnb.com/united-states/tx/austin/2018-08-14/data/calendar.csv.gz'
]

download_datasets(datasets_urls_boston, 'boston_')
download_datasets(datasets_urls_seattle, 'seattle_')
download_datasets(datasets_urls_austin, 'austin_')


datasets_cities = [
    'seattle',
    'boston',
    'austin'
]

datasets_cities = ['seattle']

File: boston_listings.csv.gz, in cache
File: boston_calendar.csv.gz, in cache
File: seattle_listings.csv.gz, in cache
File: seattle_calendar.csv.gz, in cache
File: austin_listings.csv.gz, in cache
File: austin_calendar.csv.gz, in cache


In [12]:
#Reading files and loading data into Dataframe

listings = []
calendar = []

for city in datasets_cities:
    listings += [pd.read_csv(city+'_listings.csv.gz', index_col=0)]
    calendar += [pd.read_csv(city+'_calendar.csv.gz', parse_dates=['date'])]

listings = pd.concat(listings)
calendar = pd.concat(calendar)

#calendar = dd.from_pandas(calendar, npartitions=100)

C:\Users\0xffa\AppData\Local\conda\conda\envs\Devel\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (87) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [13]:
# Data cleanning

#transform 't' or 'f' into True or False values.
calendar.available = calendar.available.apply(lambda x: x == "t")
#remove $ sign and parse the value as numeric.
calendar.price = calendar.price.apply(lambda x: float(str(x).replace("$","").replace(",","")))

calendar.dtypes

listing_id             int64
date          datetime64[ns]
available               bool
price                float64
dtype: object

In [14]:
calendar['price'].describe()

count    1.128970e+06
mean     1.707162e+02
std      1.336236e+02
min      1.000000e+01
25%      9.000000e+01
50%      1.330000e+02
75%      2.010000e+02
max      9.990000e+03
Name: price, dtype: float64

In [15]:
calendar.groupby('listing_id').count().mean()

date         365.000000
available    365.000000
price        132.913822
dtype: float64

Data per listing is related to a entire year

In [16]:
%%time
calendar['year'] = 0
calendar['month'] = 0
calendar['day'] = 0
def date_to_cols(x):
    x.year = x.date.year
    x.month = x.date.month
    x.day = x.date.day
    return x
calendar = calendar.swifter.apply(lambda x: date_to_cols(x), axis=1)


Wall time: 1min 15s


In [17]:
calendar = calendar.drop(['date'], axis=1)

In [18]:
test_calendar = calendar.copy()
test_calendar['price_per_listing'] = 0.0
test_calendar = test_calendar.fillna(0)

def apply_per_listing(df):
    df.price_per_listing = MinMaxScaler().fit_transform(df.price.values.reshape(-1, 1))
    return df
test_calendar = test_calendar.groupby('listing_id').apply(lambda x: apply_per_listing(x))

In [21]:
stats_calendar = calendar.price.describe()

In [29]:
test_calendar.filter(lambda x: x.price > stats_calendar['75%']).groupby('month')

TypeError: 'function' object is not iterable